# Maximisation couple moyen

Ce notebook propose d'optimiser le rotor d'une machine synchro-réluctante sans aimant à une paire de pôle en vue de maximiser le couple moyen.
Dans ce cas particulier, le couple moyen est directement relié à la différence de perméance magnétique du rotor dans l'axe direct d et l'axe en quadrature q.

Le problème étudié est donc la maximisation de cette différence de perméance via la différence de compliance magnétique. Il est possible de démontrer que ce problème est mal posé.

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import CSG2d, Circle, Rectangle
from copy import copy
import numpy as np
import matplotlib.pyplot as plt
from ngsolve.internal import visoptions 
visoptions.scalfunction = "Flux:0"

importing NGSolve-6.2.2105-9-g5a835126f


## 1) Définition du maillage

La première étape consiste à définir une géométrie (disque), ainsi que la finesse de maillage associée.

In [2]:
def generate_fitted_circle_mesh(N):
    geo = CSG2d()
    R=1
    x = R*2*(np.append(np.insert(np.arange(0.5,N+0.5),0,0),N)/N-0.5)
    
    circle1 = Circle( center=(0,0), radius=R, bc="left_up" ) * Rectangle( pmin=(-R,0), pmax=(0,R))
    circle2 = Circle( center=(0,0), radius=R, bc="left_bot" ) * Rectangle( pmin=(-R,-R), pmax=(0,0))
    circle3 = Circle( center=(0,0), radius=R, bc="right_bot" ) * Rectangle( pmin=(0,-R), pmax=(R,0))
    circle4 = Circle( center=(0,0), radius=R, bc="right_up" ) * Rectangle( pmin=(0,0), pmax=(R,R))
    
    materials = ["iron","air"]
    
    for i in range(len(x)-1):
        geo.Add(Rectangle( pmin=(x[i],-R), pmax=(x[i+1],R), mat = materials[i%2] ) * (circle1 + circle2 + circle3 +circle4))

    #m = geo.GenerateMesh(maxh=max([R/N,1/30])) # On doit fixer la taille du maillage sinon le volume change à cause des elts grossiers
    m = geo.GenerateMesh(maxh=1/N)
    return Mesh(m)

mesh = generate_fitted_circle_mesh(10)
Draw (mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2105-9-g5a835126f', 'mesh_dim': 2, 'order2d': 1, 'order3d': 1, 'dr…

BaseWebGuiScene

## 2) Définition du problème
Tenseur de perméabilité anisotrope homogénéisé :

$$\mu^*(\theta,\rho) = R(\theta)^T \underbrace{\begin{bmatrix} \mu_{D}(\rho) & 0 \\ 0 & \mu_{Q}(\rho) \end{bmatrix}}_{M(\rho)} R(\theta) $$

Avec $\rho$ la fraction volumique ($\simeq$ densité) et $\theta\in[-\pi/2,\pi/2]$ l'orientation privilégiée ($\theta = 0 \Rightarrow$ fibre orientée selon $\vec{x}$). $R$ est une matrice de rotation :
$$ R(\theta) = \begin{bmatrix} \cos(\theta) & - \sin(\theta) \\ \sin(\theta) & \cos(\theta) \end{bmatrix}$$

On définit aussi la perméabilité dans l'axe D (parallèle au flux):

$$\mu_{D}(\rho) = \mu_0 [\rho (\mu_r-1) + 1 ]$$

Et la perméabilité dans l'axe Q (perpendiculaire au flux):

$$\mu_{Q}(\rho) = \frac{\mu_0\mu_r}{\mu_r(1-\rho) + 1}$$


In [3]:
mu0 = 4e-7 * np.pi
mur = 5

varspace = L2(mesh)    
rho = GridFunction(varspace)
theta = GridFunction(varspace)

rho.Set(0.5)
theta.Set(np.pi/6)

def R(th):
    return CoefficientFunction( ( (cos(th),-sin(th)), (sin(th), cos(th)) ), dims = (2,2) )

def tR(th):
    return CoefficientFunction( ( (cos(th),sin(th)), (-sin(th), cos(th)) ), dims = (2,2) )

def muD(rh): 
    return mu0*(rh*(mur-1)+1)

def muQ(rh):
    return mu0*mur/(mur*(1-rh)+rh)

def M(rh):
    return CoefficientFunction( ( (muD(rh),0),(0,muQ(rh)) ), dims = (2,2) )

def mu_star(rh,th):
    return tR(th)*M(rh)*R(th)

def DrawMuStar(rh,th):
    Draw(rh*CoefficientFunction((cos(th),sin(th))),mesh,vectors = { "grid_size":20},min=0,max=1);

Le problème sous forme faible s'écrit :
$$ \int_\Omega \nabla \Phi \mu^* \nabla \phi = \int_{\partial \Omega} \Phi \beta \vec{x}|\vec{y}. \vec{n} $$

In [4]:
beta = 1;

fespace_H1 = H1(mesh, order=1)
fespace_H1.FreeDofs()[0] = False

def solvePb(rh,th):
    phi = fespace_H1.TrialFunction()
    psi = fespace_H1.TestFunction()
    K = BilinearForm(fespace_H1, symmetric=True)
    K += grad(psi)*( mu_star(rh,th)*grad(phi) )*dx
    
    l1 = LinearForm(fespace_H1)
    l1 += -psi* beta * sqrt(1-x*x)* ds(definedon=mesh.Boundaries("right_bot|left_bot"))
    l1 += psi*beta* sqrt(1-x*x)*ds(definedon=mesh.Boundaries("right_up|left_up"))

    l2 = LinearForm(fespace_H1)
    l2 += -psi*beta*sqrt(1-y*y)*ds(definedon=mesh.Boundaries("right_bot|right_up"))
    l2 += psi*beta*sqrt(1-y*y)*ds(definedon=mesh.Boundaries("left_bot|left_up"))
    
    K.Assemble()
    invK = K.mat.Inverse(inverse="sparsecholesky")
    l1.Assemble()
    l2.Assemble()
    
    phi1 = GridFunction(fespace_H1)  # solution
    phi1.vec.data =     invK * l1.vec
    phi2 = GridFunction(fespace_H1)  # solution
    phi2.vec.data =     invK * l2.vec
    
    return(phi1, phi2)

## 3) Optimisation
La compliance anisotrope s'écrit :
$$ J(\phi,\rho,\theta) = \frac{1}{2} \int_\Omega \nabla \phi .\mu^*(\rho,\theta) \nabla \phi $$

Elle admet des dérivées partielles non nulles par rapport à $\rho$ et $\theta$ :

$$ \langle \partial_{\rho} J, \Phi \rangle = \frac{1}{2} \int_\Omega \Phi . \left ( \nabla \phi .\frac{\partial\mu^*}{\partial \rho} \nabla \phi \right)$$
$$ \langle \partial_{\theta} J, \Phi \rangle = \frac{1}{2} \int_\Omega \Phi . \left ( \nabla \phi .\frac{\partial\mu^*}{\partial \theta} \nabla \phi \right)$$
$$ \langle \partial_{\phi} J, \Phi \rangle = \frac{1}{2} \int_\Omega \nabla \Phi . \left ( (\mu^* +\mu^{*T}) \nabla \phi\right) =  \int_\Omega \nabla \Phi \mu^* \nabla \phi $$

Le problème est donc auto-adjoint, mais il ne faut pas oublier les dérivées partielles par rapport à $\theta$ et $\rho$ dans l'expression de la dérivée de la compliance.

In [5]:
def compliance(phi,rh,th):
    return Integrate(grad(phi)* (mu_star(rh,th)* grad(phi)), mesh) /2

def computeGradient(phi1,phi2,rh,th):
    
    mustar = mu_star(rh,th)    
    Lag = ( grad(phi1)*(mustar*grad(phi1)) + grad(phi2)*(mustar*grad(phi2)) )*dx # FV
    Lag += ( grad(phi1)* (mustar * grad(phi1)) -  grad(phi2)* (mustar* grad(phi2)) )/2*dx # fonction objectif
    
    # dérivée par rapport à rho
    rho_test = rh.space.TestFunction()
    dLag_drho_t = LinearForm(rh.space)
    dLag_drho_t += Lag.Diff(rh,rho_test)
    dLag_drho_t.Assemble()
    dLag_drho = GridFunction(rh.space)
    dLag_drho.vec.data = dLag_drho_t.vec.data
    
    # dérivée par rapport à theta
    theta_test = th.space.TestFunction()
    dLag_dtheta_t = LinearForm(th.space)
    dLag_dtheta_t += Lag.Diff(th,theta_test)
    dLag_dtheta_t.Assemble()
    dLag_dtheta = GridFunction(th.space)
    dLag_dtheta.vec.data = dLag_dtheta_t.vec.data
    
    return dLag_drho, dLag_dtheta

def computeGradient2(phi1,phi2,rh,th):
    
    mustar = mu_star(rh,th)
    
    dmdrho = CoefficientFunction( ( ((mur-1)*mu0,0),(0,mur*mu0*(mur-1)/(mur+rh-mur*rh)**2 ) ), dims = (2,2) )
    dmudrho = tR(th) * (dmdrho * R(th))
    dmudtheta =  tR(th+np.pi/2) * ( mustar * R(th)) + tR(th) * ( mustar * R(th+np.pi/2))
    
    dJ_drho =  -grad(phi1) * (dmudrho*grad(phi1) ) + grad(phi2) * (dmudrho*grad(phi2) )
    dJ_dtheta = grad(phi1) * (dmudtheta*grad(phi1) ) - grad(phi2) * (dmudtheta*grad(phi2) )
    
    return dJ_drho, dJ_dtheta

In [ ]:
thetaList = []
rhoList = []
Jlist = []
dJdrho = GridFunction(varspace)
dJdtheta = GridFunction(varspace)
step = 0.1

for i in range(1000):
    # Résolution du problème
    
    phi1, phi2 = solvePb(rho,theta)
    
    # Calcul et normalisation du gradient
    
    dJdrho_brut, dJdtheta_brut = computeGradient2(phi1,phi2,rho,theta) 
    
    dJdrho.Set(dJdrho_brut)
    dJdrho.vec[:]= np.sign(dJdrho.vec[:].FV().NumPy())
    dJdtheta.Set(dJdtheta_brut)
    dJdtheta.vec[:]= np.sign(dJdtheta.vec[:].FV().NumPy())
    
    Jlist.append(compliance(phi1,rho,theta)-compliance(phi2,rho,theta))
    thetaList.append(copy(theta))
    rhoList.append(copy(rho))
     
    # contrôle du pas
    
    if i>0 and Jlist[-1]< Jlist[-2]:
        step = min([step*1.2,1])
    elif i>0:
        step = step/2
    
    rho.Set( rhoList[-1] - dJdrho*step )
    rho.vec[:].FV().NumPy()[rho.vec[:].FV().NumPy()<0]=0
    rho.vec[:].FV().NumPy()[rho.vec[:].FV().NumPy()>1]=1
    
    theta.Set( thetaList[-1] - dJdtheta*step*np.pi )
    theta.vec[:].FV().NumPy()[theta.vec[:].FV().NumPy()<-np.pi/2]=-np.pi/2
    theta.vec[:].FV().NumPy()[theta.vec[:].FV().NumPy()>np.pi/2]=np.pi/2
    
    if np.isnan(np.sum(rho.vec[:].FV().NumPy())) or np.isnan(np.sum(theta.vec[:].FV().NumPy())):
        break
    if step < 1e-4:
        break
    
    print(f'{Jlist[-1]} - step = {step}')

166466.84993444313 - step = 0.1
258896.82144999574 - step = 0.05
251468.8889383555 - step = 0.06
206626.9983134431 - step = 0.072
162508.3335635505 - step = 0.08639999999999999
95488.15137543128 - step = 0.10367999999999998
21211.790500099625 - step = 0.12441599999999997
28.67282228593831 - step = 0.14929919999999997


In [ ]:
DrawMuStar(rhoList[-2],thetaList[-2])

In [ ]:
Draw(rho)

In [ ]:
plt.plot(Jlist)

In [ ]:
muD(0.5)

In [ ]:
muQ(0.5)

In [ ]:
Draw(dJdrho,mesh)

In [ ]:
mustar = mu_star(rho,theta)
dmustar_dtheta = mustar.Diff(theta.space)

In [ ]:
phi1, phi2 = solvePb(rho,theta)

In [ ]:
Draw(mu_star(rho,theta*0)[0],mesh)

In [ ]:
varspace = L2(mesh, order=0)    
rho = GridFunction(varspace)
theta = GridFunction(varspace)
rho.Set(0.5)
theta.Set(0)

phi1, phi2 = solvePb(rho,theta)
dJdrho_brut, dJdtheta_brut = computeGradient(phi1,phi2,rho,theta) 
Integrate(dJdtheta_brut,mesh)

In [ ]:
varspace = L2(mesh, order=0)    
rho = GridFunction(varspace)
theta = GridFunction(varspace)
rho.Set(0.5)
theta.Set(0.1)

phi1, phi2 = solvePb(rho,theta)
dJdrho_brut, dJdtheta_brut = computeGradient(phi1,phi2,rho,theta) 
Integrate(dJdtheta_brut,mesh)

In [ ]:
varspace = L2(mesh, order=0)    
rho = GridFunction(varspace)
theta = GridFunction(varspace)
rho.Set(0.5)
theta.Set(0.2)

phi1, phi2 = solvePb(rho,theta)
dJdrho_brut, dJdtheta_brut = computeGradient(phi1,phi2,rho,theta) 
Integrate(dJdtheta_brut,mesh)

In [ ]:
varspace = L2(mesh, order=0)    
rho = GridFunction(varspace)
theta = GridFunction(varspace)
rho.Set(0)
theta.Set(np.pi/2)

phi1, phi2 = solvePb2(rho,theta)
print(Integrate(grad(phi1)* mu0* grad(phi1), mesh) /2)
print(Integrate(grad(phi2)* mu0* grad(phi2), mesh) /2)

In [ ]:
def solvePb2(rh,th):
    phi = fespace_H1.TrialFunction()
    psi = fespace_H1.TestFunction()
    mu0 = 4e-7*pi;
    K = BilinearForm(fespace_H1, symmetric=True)
    K += grad(psi)* mu0*grad(phi) *dx
    
    l1 = LinearForm(fespace_H1)
    l1 += -psi* beta * sqrt(1-x*x)* ds(definedon=mesh.Boundaries("right_bot|left_bot"))
    l1 += psi*beta* sqrt(1-x*x)*ds(definedon=mesh.Boundaries("right_up|left_up"))

    l2 = LinearForm(fespace_H1)
    l2 += -psi*beta*sqrt(1-y*y)*ds(definedon=mesh.Boundaries("right_bot|right_up"))
    l2 += psi*beta*sqrt(1-y*y)*ds(definedon=mesh.Boundaries("left_bot|left_up"))
    
    K.Assemble()
    invK = K.mat.Inverse(inverse="sparsecholesky")
    l1.Assemble()
    l2.Assemble()
    
    phi1 = GridFunction(fespace_H1)  # solution
    phi1.vec.data =     invK * l1.vec
    phi2 = GridFunction(fespace_H1)  # solution
    phi2.vec.data =     invK * l2.vec
    
    return(phi1, phi2)